In [49]:
import torch
x = torch.arange(4.0)
x.requires_grad_(True)
x.grad

In [50]:
y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [51]:
y.backward(retain_graph=True)
#注意这里的梯度，不是x对x的梯度，而是y进行反向传播后，赋值给x的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

In [52]:
#如果没有下面这一行，就会不断自动累加梯度
x.grad.zero_()

y=x.sum()
y.backward()
x.grad


tensor([1., 1., 1., 1.])

In [53]:
x.grad.zero_()
y = x* x
y.backward(torch.ones(len(x)))   #对于非标量进行backward，需要传入参数‘gradient’指定
#我们计算的目的不是微分矩阵，而是一个样本的偏导数之和
x.grad
'''
等价于y.sum().backward()
'''


'\n等价于y.sum().backward()\n'

In [57]:
x.grad.zero_()
y = x*x
# #这里我们利用detach，将u视作常数，而不与x发生关联
u = y.detach()
#这里应该可以固定参数的呀
z= u*x
z.sum().backward()
x.grad
#给我整不会了


tensor([0., 1., 4., 9.])

In [55]:
x.grad.zero_()
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

对于复杂控制流的求导：

In [67]:
def f(a):
    b= a*2
    while b.norm() < 1000:
        b=  b * 2
    if b.sum() > 0:
        c = b
    else: 
        c = 100 * b 
    return c 
#一个记录梯度的标量a
a =  torch.randn(size = (),requires_grad = True)
d = f(a)
d.backward()
a.grad,d,a,a.grad==d/a


(tensor(102400.),
 tensor(-162351.7188, grad_fn=<MulBackward0>),
 tensor(-1.5855, requires_grad=True),
 tensor(True))

说明:

1.pytorch自动累计梯度：批量情况

2.为什么求导不喜欢对向量：因为标量loss简单，好球

3.如果么有进行backward，当然就不会对grad进行梯度赋值啦，这是很贵的一件事。

4.计算的时候都是有向图，树状图，不喜欢环状图

